In [ ]:
pip install --upgrade ultralytics

In [ ]:
# Cuts the dataset into 5 folds (images only, no labels)
import os
import shutil
from sklearn.model_selection import StratifiedKFold
from pathlib import Path

# Parameters: adjust these paths as needed
dataset_dir = r"C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage"
output_dir = r"C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds"
n_splits = 5
random_state = 42
copy_files = True  # Set to False to use symlinks instead of copying

# Gather all samples and class labels
train_dir = Path(dataset_dir) / 'train'
test_dir = Path(dataset_dir) / 'test'
samples = []  # (img_path, class_name) - Removed label_path
class_names = []
for cls_dir in sorted(train_dir.iterdir()):
    if not cls_dir.is_dir():
        continue
    cls = cls_dir.name
    class_names.append(cls)
    for img_path in (cls_dir / 'images').glob('*.*'):
        samples.append((img_path, cls))  # Removed lbl_path

# Prepare arrays for stratification
image_paths = [s[0] for s in samples]
y = [class_names.index(s[1]) for s in samples]  # Changed index to use the class_name from the simplified samples

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
summary = {}

# Create folds
for fold_idx, (train_idx, val_idx) in enumerate(skf.split(image_paths, y), start=1):
    fold_base = Path(output_dir) / f"fold_{fold_idx}"
    summary[fold_idx] = {'train': len(train_idx), 'val': len(val_idx), 'test': 0}

    # Train/val splits
    for split, idxs in [('train', train_idx), ('val', val_idx)]:
        for cls in class_names:
            (fold_base / split / cls / 'images').mkdir(parents=True, exist_ok=True)
        for i in idxs:
            img_src, cls_name = image_paths[i], samples[i][1]  # Removed lbl_src
            img_dst = fold_base / split / cls_name / 'images' / img_src.name
            if copy_files:
                shutil.copy(img_src, img_dst)
            else:
                os.symlink(img_src.resolve(), img_dst)

    # Copy test set unchanged
    if test_dir.exists():
        dst_test = fold_base / 'test'
        if dst_test.exists():
            shutil.rmtree(dst_test)
        shutil.copytree(test_dir, dst_test)
        test_images_dir = dst_test / 'images'
        summary[fold_idx]['test'] = len(list(test_images_dir.glob('*.*')))

# Print out summary to verify
print("K-fold generation summary:")
for fold, counts in summary.items():
    print(f"Fold {fold}: train={counts['train']}, val={counts['val']}, test={counts['test']}")

In [6]:
import os
import gc
import torch
from ultralytics import YOLO

# ---------------- CONFIG ----------------
BASE_DIR = r"C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds"  # root path where fold_1, …, fold_5 live
MODEL_FILE = r"models\yolo11n-cls.pt"  # pretrained classification model
FOLDS = 5
EPOCHS = 100
IMGSZ = 640
BATCH = 16  # reduced for lower VRAM usage
PATIENCE = 25  # Reduced patience for earlier stopping
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MEM_FRACTION = 0.6  # 60% GPU memory cap
# ----------------------------------------

print(f"Using device: {DEVICE}\n")

if DEVICE == "cuda":
    torch.cuda.set_per_process_memory_fraction(MEM_FRACTION, device=0)

for fold in range(1, FOLDS + 1):
    gc.collect()
    torch.cuda.empty_cache()

    fold_dir = os.path.join(BASE_DIR, f"fold_{fold}")
    if not os.path.isdir(os.path.join(fold_dir, 'train')):
        raise FileNotFoundError(f"{fold_dir}/train not found")

    print(f"\n=== Training Fold {fold}/{FOLDS} on folder {fold_dir} ===")
    model = YOLO(MODEL_FILE)
    model.train(
        data=fold_dir,
        task='classify',
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        patience=PATIENCE,
        device=DEVICE,
        project=fold_dir,
        name='results',
    )

    print(f"--- Fold {fold} complete. Best weights in {fold_dir}/results ---")


Using device: cuda


=== Training Fold 1/5 on folder C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_1 ===
New https://pypi.org/project/ultralytics/8.3.134 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.133  Python-3.9.21 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_1, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.0

train: Scanning C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_1\train... 695 images, 0 corrupt: 100%|██████████| 695/695 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.40.1 ms, read: 294.045.3 MB/s, size: 186.9 KB)


val: Scanning C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_1\val... 174 images, 0 corrupt: 100%|██████████| 174/174 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_1\results3
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

     75/100      2.05G     0.0149          8        640: 100%|██████████| 44/44 [00:03<00:00, 12.02it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 6/6 [00:00<00:00, 20.30it/s]

                   all      0.983          1
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 50, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



75 epochs completed in 0.095 hours.
Optimizer stripped from C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_5\results2\weights\last.pt, 3.2MB
Optimizer stripped from C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_5\results2\weights\best.pt, 3.2MB

Validating C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_5\results2\weights\best.pt...
Ultralytics 8.3.133  Python-3.9.21 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n-cls summary (fused): 47 layers, 1,532,429 parameters, 0 gradients, 3.2 GFLOPs
train: C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_5\train... found 696 images in 5 classes  
val: C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_5\val... found 173 images in 5 classes  
test: C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_5\test... found 43 images in 5 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 6/6 [00:00<00:00, 11.19it/s]


                   all      0.994          1
Speed: 0.5ms preprocess, 2.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_5\results2
--- Fold 5 complete. Best weights in C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds\fold_5/results ---


In [ ]:
from ultralytics import YOLO
import torch

BASE_DIR   = r"C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage"
MODEL_FILE = r"models\yolo11n-cls.pt"

# Train on ROAD_DAMAGE (which now has train/ and val/ with direct images)
model = YOLO(MODEL_FILE)
model.train(
    data     = BASE_DIR,
    task     = 'classify',
    epochs   = 100,
    imgsz    = 640,
    batch    = 8,
    patience = 5,                 # monitors REAL val/ now
    device   = 'cuda' if torch.cuda.is_available() else 'cpu',
    project  = BASE_DIR,
    name     = 'single_train',
)


In [7]:
import os
import pandas as pd
import shutil

# --- CONFIGURATION ---
BASE_DIR = r"C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\folds"
RESULTS_SUBPATH = os.path.join("results", "results.csv")
WEIGHTS_SUBPATH = os.path.join("results", "weights", "best.pt")
FINAL_MODEL_DIR = os.path.abspath(os.path.join(BASE_DIR, os.pardir, "final_model"))

# --- MAIN LOGIC ---
def find_all_folds(base_dir):
    """
    Read each fold's results.csv, extract key metrics, and return a summary DataFrame.
    """
    fold_dirs = [d for d in os.listdir(base_dir)
                 if os.path.isdir(os.path.join(base_dir, d)) and d.lower().startswith('fold_')]
    records = []

    for fold in sorted(fold_dirs):
        csv_path = os.path.join(base_dir, fold, RESULTS_SUBPATH)
        if not os.path.exists(csv_path):
            continue

        df = pd.read_csv(csv_path)
        # Identify columns
        acc_col = next((c for c in df.columns if 'accuracy' in c.lower()), None)
        train_loss_col = next((c for c in df.columns if 'train/loss' in c.lower()), None)
        val_loss_col = next((c for c in df.columns if 'val/loss' in c.lower()), None)
        lr_col = next((c for c in df.columns if c.lower().startswith('lr/')), None)

        if acc_col is None:
            continue

        # Best epoch by accuracy
        best_idx = df[acc_col].idxmax()
        best = df.loc[best_idx]

        records.append({
            'Fold': fold,
            'Epoch': int(best.get('epoch', best_idx + 1)),
            'Accuracy': best[acc_col],
            'Train Loss': best.get(train_loss_col, pd.NA),
            'Val Loss': best.get(val_loss_col, pd.NA),
            'Loss Gap': abs(best.get(train_loss_col, 0) - best.get(val_loss_col, 0)),
            'Learning Rate': best.get(lr_col, pd.NA)
        })

    if not records:
        print("⚠️ No valid folds found.")
        return pd.DataFrame()

    results_df = pd.DataFrame(records)
    # Ranking
    results_df['Accuracy Rank'] = results_df['Accuracy'].rank(ascending=False)
    results_df['Loss Gap Rank'] = results_df['Loss Gap'].rank(ascending=True)
    results_df['Overall Score'] = results_df['Accuracy Rank'] + results_df['Loss Gap Rank']
    return results_df


def select_and_copy_best(results_df, base_dir, final_dir):
    """
    Select the best fold based on Overall Score and copy its best.pt to final_dir.
    """
    if results_df.empty:
        return

    # Display all folds
    print("\n📊 Cross-Validation Summary:")
    print(results_df.to_string(index=False))

    # Pick best
    best_row = results_df.loc[results_df['Overall Score'].idxmin()]
    print(f"\n🎯 Best Fold: {best_row['Fold']} at Epoch {best_row['Epoch']}")

    # Copy model
    os.makedirs(final_dir, exist_ok=True)
    source = os.path.join(base_dir, best_row['Fold'], WEIGHTS_SUBPATH)
    dest = os.path.join(final_dir, "best_fold.pt")
    if os.path.exists(source):
        shutil.copy(source, dest)
        print(f"📦 Copied model to {dest}")
    else:
        print(f"❌ Model file missing at {source}")


if __name__ == '__main__':
    df_summary = find_all_folds(BASE_DIR)
    select_and_copy_best(df_summary, BASE_DIR, FINAL_MODEL_DIR)



📊 Cross-Validation Summary:
  Fold  Epoch  Accuracy  Train Loss  Val Loss  Loss Gap  Learning Rate  Accuracy Rank  Loss Gap Rank  Overall Score
fold_1      5   0.97701     0.18850   0.12122   0.06728       0.001067            5.0            3.0            8.0
fold_2     13   0.98851     0.16576   0.04024   0.12552       0.000979            2.5            4.0            6.5
fold_3     13   1.00000     0.15628   0.01903   0.13725       0.000979            1.0            5.0            6.0
fold_4     14   0.98851     0.11902   0.05456   0.06446       0.000968            2.5            2.0            4.5
fold_5     25   0.98266     0.06221   0.04422   0.01799       0.000847            4.0            1.0            5.0

🎯 Best Fold: fold_4 at Epoch 14
📦 Copied model to C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\final_model\best_fold.pt


In [4]:
# CONFUSION MATRIX

from ultralytics import YOLO
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os

# 1. Reload your trained classification weights
weights = "Road_Damage/final_model/best_fold.pt"

model   = YOLO(weights)

# 2. Grab and extend the model's names dict
names = model.names.copy()
no_damage_idx = max(names.keys()) + 1
names[no_damage_idx] = "no_damage"

# 3. Validation setup
val_root = "Road_Damage/test"
folders  = [
    "potholes",
    "crack_issues",
    "alligator_crack_issues",
    "ravelling",
    "open_manhole"
]

# 4. Invert for lookup
name_to_idx = {v: k for k, v in names.items()}

# 5. Choose a confidence threshold below which we call it "no_damage"
CONF_THRESH = 0.3

y_true, y_pred = [], []

for folder in folders:
    true_idx = name_to_idx[folder]
    img_dir  = os.path.join(val_root, folder, "images")
    for fn in os.listdir(img_dir):
        if not fn.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        path = os.path.join(img_dir, fn)
        y_true.append(true_idx)

        # Run classification
        res = model(path, imgsz=640)[0]

        # Use top1 and top1conf directly
        top_idx  = int(res.probs.top1)
        top_conf = float(res.probs.top1conf)

        # Assign no_damage if confidence is too low
        if top_conf < CONF_THRESH:
            pred_idx = no_damage_idx
        else:
            pred_idx = top_idx

        y_pred.append(pred_idx)
# 6. Print classification report (include labels for all classes)
sorted_idx = sorted(name_to_idx.values())  # e.g. [0,1,2,3,4,5,6]

print("\n📊 Classification Report:")
print(classification_report(
    y_true,
    y_pred,
    labels=sorted_idx,
    target_names=[names[i] for i in sorted_idx],
    zero_division=0
))

# 7. Build & save the confusion matrix
cm = confusion_matrix(
    y_true,
    y_pred,
    labels=sorted_idx
)
disp = ConfusionMatrixDisplay(
    cm,
    display_labels=[names[i] for i in sorted_idx]
)

fig, ax = plt.subplots(figsize=(8, 7))
disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")
plt.title("Validation Confusion Matrix (incl. no_damage)")
plt.tight_layout()

out_path = "confusion_matrix_with_no_damage.png"
plt.savefig(out_path, dpi=150)
print(f"Saved confusion matrix to {out_path}")
plt.close(fig)



image 1/1 C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\test\potholes\images\4 -  Potholes (12).jpg: 640x640 potholes 1.00, crack_issues 0.00, ravelling 0.00, alligator_crack_issues 0.00, open_manhole 0.00, 26.2ms
Speed: 28.1ms preprocess, 26.2ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\test\potholes\images\4 -  Potholes (13).jpg: 640x640 potholes 1.00, crack_issues 0.00, ravelling 0.00, alligator_crack_issues 0.00, open_manhole 0.00, 26.3ms
Speed: 7.5ms preprocess, 26.3ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov11\Road_Damage\test\potholes\images\4 -  Potholes (14).jpg: 640x640 potholes 1.00, crack_issues 0.00, ravelling 0.00, open_manhole 0.00, alligator_crack_issues 0.00, 24.2ms
Speed: 6.7ms preprocess, 24.2ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\jazzb\ImageDetection-Yolov11\Ro

In [ ]:
# CONFUSION MATRIX

from ultralytics import YOLO
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os

# 1. Reload your trained classification weights
weights = "best_fold.pt"

model   = YOLO(weights)

# 2. Grab and extend the model's names dict
names = model.names.copy()
no_damage_idx = max(names.keys()) + 1
names[no_damage_idx] = "no_damage"

# 3. Validation setup
val_root = "Road_Damage/test"
folders  = [
    "potholes",
    "crack_issues",
    "alligator_crack_issues",
    "ravelling",
    "open_manhole"
]

# 4. Invert for lookup
name_to_idx = {v: k for k, v in names.items()}

# 5. Choose a confidence threshold below which we call it "no_damage"
CONF_THRESH = 0.3

y_true, y_pred = [], []

for folder in folders:
    true_idx = name_to_idx[folder]
    img_dir  = os.path.join(val_root, folder, "images")
    for fn in os.listdir(img_dir):
        if not fn.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        path = os.path.join(img_dir, fn)
        y_true.append(true_idx)

        # Run classification
        res = model(path, imgsz=640)[0]

        # Use top1 and top1conf directly
        top_idx  = int(res.probs.top1)
        top_conf = float(res.probs.top1conf)

        # Assign no_damage if confidence is too low
        if top_conf < CONF_THRESH:
            pred_idx = no_damage_idx
        else:
            pred_idx = top_idx

        y_pred.append(pred_idx)
# 6. Print classification report (include labels for all classes)
sorted_idx = sorted(name_to_idx.values())  # e.g. [0,1,2,3,4,5,6]

print("\n📊 Classification Report:")
print(classification_report(
    y_true,
    y_pred,
    labels=sorted_idx,
    target_names=[names[i] for i in sorted_idx],
    zero_division=0
))

# 7. Build & save the confusion matrix
cm = confusion_matrix(
    y_true,
    y_pred,
    labels=sorted_idx
)
disp = ConfusionMatrixDisplay(
    cm,
    display_labels=[names[i] for i in sorted_idx]
)

fig, ax = plt.subplots(figsize=(8, 7))
disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")
plt.title("Validation Confusion Matrix (incl. no_damage)")
plt.tight_layout()

out_path = "confusion_matrix_with_no_damage.png"
plt.savefig(out_path, dpi=150)
print(f"Saved confusion matrix to {out_path}")
plt.close(fig)


In [24]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# 1. Load model
model = YOLO("best.pt")
model.eval()

# 2. Parameters
image_path   = "testing/4.jpg"
patch_size   = 256
stride        = 128
conf_thresh   = 0.6
max_patches   = 4   # limit for “other” damages

# 3. Read & classify the whole image
img_bgr = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
whole_res = model(img_rgb, imgsz=640)[0]
primary_idx  = int(whole_res.probs.top1)
primary_conf = float(whole_res.probs.top1conf)
primary_lbl  = model.names.get(primary_idx, str(primary_idx))
print(f"Primary damage → {primary_lbl} ({primary_conf:.2f})")

# 4. Sliding window & collect detections **excluding** primary
h, w, _ = img_rgb.shape
all_dets = []
for y in range(0, h - patch_size + 1, stride):
    for x in range(0, w - patch_size + 1, stride):
        patch = img_rgb[y:y+patch_size, x:x+patch_size]
        res   = model(patch, imgsz=patch_size)[0]
        conf  = float(res.probs.top1conf)
        cls   = int(res.probs.top1)
        lbl   = model.names.get(cls, str(cls))
        # only keep if above threshold and not the primary damage
        if conf >= conf_thresh and lbl != primary_lbl:
            all_dets.append({"label": lbl, "conf": conf, "coords": (x, y)})

# 5. Keep best patch per remaining class
best_per_class = {}
for det in all_dets:
    lbl = det["label"]
    if lbl not in best_per_class or det["conf"] > best_per_class[lbl]["conf"]:
        best_per_class[lbl] = det

# 6. Sort & take up to max_patches “other” damages
other_dets = sorted(best_per_class.values(), key=lambda d: d["conf"], reverse=True)[:max_patches]
other_labels = [d["label"] for d in other_dets]

# 7. Report “other possible damages”
if other_labels:
    print(f"Other possible damages (top {len(other_labels)}): {', '.join(other_labels)}")
else:
    print("No other damages detected above threshold.")

# 8. Visualization: annotate primary (full-image) and secondary patches
vis = img_rgb.copy()
# a) Draw a banner for primary damage
banner = f"Primary: {primary_lbl} ({primary_conf:.2f})"
cv2.rectangle(vis, (0, 0), (w, 30), (0, 0, 255), -1)
cv2.putText(vis, banner, (5, 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

# b) Draw boxes for each “other” patch
for det in other_dets:
    x, y   = det["coords"]
    label  = f"{det['label']} ({det['conf']:.2f})"
    # patch box
    cv2.rectangle(vis, (x, y), (x+patch_size, y+patch_size), (255, 0, 0), 2)
    # label background
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
    cv2.rectangle(vis, (x, y-th-4), (x+tw+4, y), (255, 0, 0), -1)
    # label text
    cv2.putText(vis, label, (x+2, y-2),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

# 9. Display result
plt.figure(figsize=(12, 10))
plt.imshow(vis)
plt.axis("off")
plt.title("Primary damage (top banner) + Other possible damages")
plt.tight_layout()
plt.show()



0: 640x640 alligator_crack_issues 0.49, ravelling 0.48, crack_issues 0.02, open_manhole 0.00, potholes 0.00, 25.7ms
Speed: 25.5ms preprocess, 25.7ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
Primary damage → alligator_crack_issues (0.49)

0: 640x640 open_manhole 0.98, crack_issues 0.01, potholes 0.00, ravelling 0.00, alligator_crack_issues 0.00, 25.1ms
Speed: 8.6ms preprocess, 25.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 open_manhole 0.96, crack_issues 0.04, potholes 0.00, alligator_crack_issues 0.00, ravelling 0.00, 25.9ms
Speed: 9.0ms preprocess, 25.9ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 open_manhole 0.89, crack_issues 0.10, potholes 0.01, alligator_crack_issues 0.00, ravelling 0.00, 25.3ms
Speed: 10.2ms preprocess, 25.3ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 open_manhole 0.80, crack_issues 0.20, alligator_crack_issues 0.00, potholes 0.00, r

<Figure size 1200x1000 with 1 Axes>

In [ ]:
from ultralytics import YOLO
from sklearn.metrics import auc as sklearn_auc

from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score
)
from sklearn.exceptions import UndefinedMetricWarning
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings

# 1. Load trained classification model
weights = "Road_Damage/single_train/weights/best.pt"
model = YOLO(weights)

# 2. Get class names
names = model.names.copy()

# 3. Set up validation folders
val_root = "Road_Damage/test"
folders = [
    "potholes",
    "crack_issues",
    "alligator_crack_issues",
    "ravelling",
    "open_manhole"
]
name_to_idx = {v: k for k, v in names.items()}

# 4. Evaluate model
y_true, y_pred, y_probs = [], [], []

for folder in folders:
    true_idx = name_to_idx[folder]
    img_dir = os.path.join(val_root, folder, "images")

    for fn in os.listdir(img_dir):
        if not fn.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        path = os.path.join(img_dir, fn)
        y_true.append(true_idx)

        res = model(path, imgsz=640)[0]
        top_idx = int(res.probs.top1)

        # Collect raw prediction and probs
        y_pred.append(top_idx)
        y_probs.append(res.probs.data.tolist())

# 5. Sorted class indices and labels
sorted_idx = sorted(set(y_true + y_pred))
label_names = [names[i] for i in sorted_idx]

# 6. Print classification report
print("\n📊 Classification Report:")
print(classification_report(
    y_true,
    y_pred,
    labels=sorted_idx,
    target_names=label_names,
    zero_division=0
))

# 7. Confusion Matrix
cm = confusion_matrix(y_true, y_pred, labels=sorted_idx)
disp = ConfusionMatrixDisplay(cm, display_labels=label_names)
fig, ax = plt.subplots(figsize=(9, 7))
disp.plot(ax=ax, cmap="Blues", xticks_rotation="vertical")
plt.title("Confusion Matrix (Only Damage Classes)")
plt.tight_layout()
plt.savefig("confusion_matrix_no_no_damage.png", dpi=150)
print("✅ Saved confusion matrix to 'confusion_matrix_no_no_damage.png'")
plt.show()

# 8. ROC-AUC (One-vs-Rest)
print("\n📈 ROC-AUC Scores (One-vs-Rest):")
y_true_bin = np.zeros((len(y_true), len(sorted_idx)))
for i, cls in enumerate(y_true):
    y_true_bin[i][sorted_idx.index(cls)] = 1

y_probs_trimmed = np.array(y_probs)[:, sorted_idx]

try:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UndefinedMetricWarning)

        per_class_auc = roc_auc_score(
            y_true_bin,
            y_probs_trimmed,
            average=None,
            multi_class="ovr"
        )
        macro_auc = roc_auc_score(
            y_true_bin,
            y_probs_trimmed,
            average="macro",
            multi_class="ovr"
        )

        print(f"{'Class':<25}  ROC-AUC")
        print("-" * 38)
        for i, auc in zip(sorted_idx, per_class_auc):
            print(f"{names[i]:<25}  {auc:.3f}")
        print("-" * 38)
        print(f"{'Macro average':<25}  {macro_auc:.3f}")

except ValueError as e:
    print(f"⚠️ ROC-AUC could not be computed: {e}")

# 9. Note on unsupported metrics
print("\n📉 Plotting ROC Curves per class...")
fig, ax = plt.subplots(figsize=(8, 6))

for i, class_idx in enumerate(sorted_idx):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs_trimmed[:, i])
    roc_auc = sklearn_auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=2, label=f"{names[class_idx]} (AUC = {roc_auc:.2f})")

ax.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random")

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curves (One-vs-Rest)")
ax.legend(loc="lower right", fontsize="small")
plt.tight_layout()
plt.savefig("roc_curves_multiclass.png", dpi=150)
print("✅ ROC curve saved as 'roc_curves_multiclass.png'")
plt.show()